# **Extracting Data From API**

In [40]:
import requests
import pandas as pd
import json
# Example: Get 1000 recent drug events
url = "https://api.fda.gov/drug/event.json?limit=1000"

response = requests.get(url)
data = response.json()
results = data["results"]

# Normalize JSON into a DataFrame
df = pd.json_normalize(results)

# **Creating DataFrame from the JSON Data**

In [42]:
# Step 3: Extract nested fields safely
def extract_drug_name(drugs):
    if isinstance(drugs, list) and drugs:
        return drugs[0].get("medicinalproduct")
    return None

def extract_reaction(reactions):
    if isinstance(reactions, list) and reactions:
        return reactions[0].get("reactionmeddrapt")
    return None

# Step 4: Create cleaned columns
df['drug_name'] = df['patient.drug'].apply(extract_drug_name)
df['reaction'] = df['patient.reaction'].apply(extract_reaction)

# Map patientsex codes to labels
sex_map = {
    "1": "Male",
    "2": "Female",
    "0": "Unknown",
    "null": None
}
df['patient_sex'] = df['patient.patientsex'].astype(str).map(sex_map)

# Clean and format date
df['receivedate'] = pd.to_datetime(df['receivedate'], format='%Y%m%d', errors='coerce')

# Convert age to numeric
df['patient_age'] = pd.to_numeric(df['patient.patientonsetage'], errors='coerce')

# Step 5: Final cleaned dataframe
cleaned_df = df[[
    'safetyreportid',
    'receivedate',
    'patient_age',
    'patient_sex',
    'drug_name',
    'reaction',
    'seriousnessdeath'
]].copy()

cleaned_df.rename(columns={
    'safetyreportid': 'report_id',
    'seriousnessdeath': 'outcome'
}, inplace=True)

# Preview the cleaned data
cleaned_df.head()

,report_id,receivedate,patient_age,patient_sex,drug_name,reaction,outcome
0,5801206-7,2008-07-07,26.0,Male,DURAGESIC-100,DRUG ADMINISTRATION ERROR,1
1,10003300,2014-03-06,77.0,Female,BONIVA,Vomiting,NaN
2,10003301,2014-02-28,NaN,Female,IBUPROFEN,Dyspepsia,NaN
3,10003302,2014-03-12,NaN,Male,LYRICA,Drug ineffective,NaN
4,10003304,2014-03-12,NaN,Female,DOXYCYCLINE HYCLATE,Drug hypersensitivity,NaN


# **Cleaning & Transformation of Data**

In [44]:
cleaned_df.isna().sum()

report_id        0
receivedate      0
patient_age    341
patient_sex      4
drug_name        0
reaction         0
outcome        913
dtype: int64

In [46]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   report_id    1000 non-null   object        
 1   receivedate  1000 non-null   datetime64[ns]
 2   patient_age  659 non-null    float64       
 3   patient_sex  996 non-null    object        
 4   drug_name    1000 non-null   object        
 5   reaction     1000 non-null   object        
 6   outcome      87 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 54.8+ KB


In [48]:
import pandas as pd

# Convert 'patient_age' to int (after filling missing with -1)
cleaned_df['patient_age'] = cleaned_df['patient_age'].fillna(-1).astype(int)

# Handle 'outcome': convert missing to 0 (means not a death report)
cleaned_df['outcome'] = cleaned_df['outcome'].fillna(0).astype(int)

# If needed, ensure all string columns are actually strings
cleaned_df['report_id'] = cleaned_df['report_id'].astype(str)
cleaned_df['patient_sex'] = cleaned_df['patient_sex'].fillna("Unknown").astype(str)
cleaned_df['drug_name'] = cleaned_df['drug_name'].astype(str)
cleaned_df['reaction'] = cleaned_df['reaction'].astype(str)

In [50]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   report_id    1000 non-null   object        
 1   receivedate  1000 non-null   datetime64[ns]
 2   patient_age  1000 non-null   int32         
 3   patient_sex  1000 non-null   object        
 4   drug_name    1000 non-null   object        
 5   reaction     1000 non-null   object        
 6   outcome      1000 non-null   int32         
dtypes: datetime64[ns](1), int32(2), object(4)
memory usage: 47.0+ KB


In [52]:
cleaned_df.isna().sum()

report_id      0
receivedate    0
patient_age    0
patient_sex    0
drug_name      0
reaction       0
outcome        0
dtype: int64

In [54]:
cleaned_df.duplicated().sum()

0

In [56]:
cleaned_df.patient_age.unique()

array([26, 77, -1, 48, 68, 65, 59, 42, 66, 76, 43, 81, 82, 73, 46, 71, 61,
       57, 29, 53, 55, 79, 50, 38, 80, 62, 34, 69, 22, 54, 64, 88, 93, 51,
       52, 60, 10, 41, 31, 32, 70, 78, 49, 67, 63, 75, 72, 19, 44, 25, 56,
       18, 86, 47, 45, 30, 74, 36, 28, 37, 84, 35, 33, 40, 39, 58,  4, 83,
       24, 11, 16,  3,  9,  2, 92,  7, 23, 90,  8, 87, 85, 17, 21, 91, 89,
       14, 20, 27, 15,  6,  5, 13, 12])

---
## Column Descriptions – FDA Adverse Drug Event Dataset

The following are the columns used in the cleaned dataset after extracting and transforming JSON data from the FDA Drug Event API:

| Column Name   | Description |
|---------------|-------------|
| `report_id`   | Unique identifier for the adverse event report |
| `receivedate` | Date when the report was received by the FDA (in YYYY-MM-DD format) |
| `patient_age` | Age of the patient at the time of the event. If unknown, filled with -1 |
| `patient_sex` | Gender of the patient (Male / Female / Unknown) |
| `drug_name`   | Name of the drug suspected to have caused the adverse reaction |
| `reaction`    | Medical term describing the adverse effect reported (e.g., nausea, rash, etc.) |
| `outcome`     | Indicates whether death was reported (1 = death, 0 = no death reported or unknown) |

**Note on Missing Values:**
- `patient_age`: Missing values were filled with `-1` to indicate unknown age.
- `outcome`: Originally had nulls; filled with `0` to mean no death reported.
- `patient_sex`: Filled missing values with `"Unknown"` string.

This cleaned structure ensures the dataset is ready for SQL-based analysis, aggregation, and reporting.


In [62]:
cleaned_df.to_csv("cleaned_fda_drug_events.csv", index=False)

---
# **Connecting to MySQL**

In [66]:
import mysql.connector
from sqlalchemy import create_engine

# ✅ Replace with your MySQL credentials
user = "root"
password = "6541"
host = "127.0.0.1"
database = "fda_drug_analysis"

# ✅ Use SQLAlchemy to bulk insert the cleaned DataFrame
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# ✅ Assuming your DataFrame is called cleaned_df
cleaned_df.to_sql(name='drug_events', con=engine, if_exists='replace', index=False)

print("✅ Data inserted successfully into MySQL!")

✅ Data inserted successfully into MySQL!


<hr style="border: 2px solid orange;">
<hr style="border: 3px dashed #00aaff;">